In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"aarav20","key":"121784cd4910f61cb309ca77250bf290"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle datasets download -d akash2907/bird-species-classification

 99% 1.36G/1.37G [00:07<00:00, 213MB/s]
100% 1.37G/1.37G [00:07<00:00, 204MB/s]


In [ ]:
!unzip bird-species-classification.zip

Archive:  bird-species-classification.zip
  inflating: test_data/test_data/blasti/DSC_6396.jpg  
  inflating: test_data/test_data/blasti/DSC_6397.jpg  
  inflating: test_data/test_data/blasti/DSC_6398.jpg  
  inflating: test_data/test_data/blasti/DSC_6399.jpg  
  inflating: test_data/test_data/blasti/DSC_6400.jpg  
  inflating: test_data/test_data/blasti/DSC_6401.jpg  
  inflating: test_data/test_data/blasti/DSC_6402.jpg  
  inflating: test_data/test_data/blasti/DSC_6403.jpg  
  inflating: test_data/test_data/blasti/DSC_6405.jpg  
  inflating: test_data/test_data/blasti/DSC_6406.jpg  
  inflating: test_data/test_data/blasti/DSC_6407.jpg  
  inflating: test_data/test_data/blasti/DSC_6408.jpg  
  inflating: test_data/test_data/blasti/DSC_6409.jpg  
  inflating: test_data/test_data/blasti/DSC_6410.jpg  
  inflating: test_data/test_data/blasti/DSC_6411.jpg  
  inflating: test_data/test_data/bonegl/DSC_4587.jpg  
  inflating: test_data/test_data/bonegl/DSC_4588.jpg  
  inflating: test_data/

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers


image_height = 224
image_width = 224
batch_size = 32

train_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_data_gen.flow_from_directory(
    '/content/train_data/train_data',
    target_size=(image_height, image_width),
    batch_size=batch_size,
    subset='training',
    class_mode='categorical'
)

validation_generator = train_data_gen.flow_from_directory(
    '/content/train_data/train_data',
    target_size=(image_height, image_width),
    batch_size=batch_size,
    subset='validation',
    class_mode='categorical'
)

Found 124 images belonging to 16 classes.
Found 26 images belonging to 16 classes.


In [ ]:
num_classes = 16
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # num_classes is the number of bird species
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_generator, validation_data=validation_generator, epochs=20)

Epoch 1/20
4/4 [==============================] - 50s 10s/step - loss: 3.9242 - accuracy: 0.0726 - val_loss: 2.7436 - val_accuracy: 0.1538
Epoch 2/20
4/4 [==============================] - 37s 9s/step - loss: 2.7111 - accuracy: 0.1532 - val_loss: 2.6888 - val_accuracy: 0.2308
Epoch 3/20
4/4 [==============================] - 37s 10s/step - loss: 2.6416 - accuracy: 0.1694 - val_loss: 2.5543 - val_accuracy: 0.1923
Epoch 4/20
4/4 [==============================] - 37s 10s/step - loss: 2.4496 - accuracy: 0.1855 - val_loss: 2.4737 - val_accuracy: 0.3462
Epoch 5/20
4/4 [==============================] - 36s 9s/step - loss: 2.3441 - accuracy: 0.2339 - val_loss: 2.5177 - val_accuracy: 0.3077
Epoch 6/20
4/4 [==============================] - 36s 10s/step - loss: 2.2072 - accuracy: 0.2581 - val_loss: 2.4630 - val_accuracy: 0.3846
Epoch 7/20
4/4 [==============================] - 37s 10s/step - loss: 2.0407 - accuracy: 0.3226 - val_loss: 2.6093 - val_accuracy: 0.4231
Epoch 8/20
4/4 [=============

In [ ]:
model.save('birdClassification1.h5')

Model Tuning

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

In [ ]:
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.models import Sequential

In [ ]:
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(62, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # num_classes is the number of bird species
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stop = EarlyStopping(monitor='accuracy',patience=5)

In [ ]:
model.fit(train_generator, validation_data=validation_generator, epochs=20,callbacks=early_stop)

Epoch 1/20
4/4 [==============================] - 46s 11s/step - loss: 8.9023 - accuracy: 0.0968 - val_loss: 2.9704 - val_accuracy: 0.0385
Epoch 2/20
2/4 [==============>...............] - ETA: 12s - loss: 6.6863 - accuracy: 0.2833

KeyboardInterrupt: ignored